In [1]:
import networkx as nx
import copy

In [2]:
G = nx.DiGraph()

G.add_edges_from(
    [
        (1, 2),
        (1, 4),
        (2, 3),
        (2, 5),
        (3, 8),
        (4, 5),
        (5, 1),
        (5, 3),
        (6, 5),
        (7, 5),
        (7, 8),
        (8, 6)
    ]
)

In [3]:
l = G.out_edges(5)
ln = [i[1] for i in l]

print(l)
print(ln)

[(5, 1), (5, 3)]
[1, 3]


In [4]:
def contractGraphNode(G, v):
    
    # Gn = copy.deepcopy(G)
    Gn = nx.DiGraph(G)
    
    e = Gn.in_edges(v)
    en = [i[0] for i in e]
    
    l = Gn.out_edges(v)
    ln = [i[1] for i in l]
    
    for j in en:
        for k in ln:
            # Delete the loop vertex.
            if j == k:
                Gn.remove_node(j)
            else:
                Gn.add_edge(j, k)
    
    Gn.remove_node(v)
    
    return Gn

In [5]:
print(type(3))

<class 'int'>


In [6]:
def contractGraph(G, D):
    
    # Gn = copy.deepcopy(G)
    Gn = nx.DiGraph(G)
    
    if type(D) == type(3):
        return contractGraphNode(G, D)
    
    elif len(D) == 0:
        return Gn
    
    else:
        t = D.pop()
        return contractGraph(contractGraphNode(Gn, t), D)


In [7]:
print(G.edges)

[(1, 2), (1, 4), (2, 3), (2, 5), (4, 5), (3, 8), (5, 1), (5, 3), (8, 6), (6, 5), (7, 5), (7, 8)]


In [8]:
print(contractGraphNode(G, 5).edges)

[(1, 2), (1, 4), (2, 3), (2, 1), (4, 1), (4, 3), (3, 8), (8, 6), (6, 1), (6, 3), (7, 8), (7, 1), (7, 3)]


In [9]:
print(contractGraph(G, (5)).edges)

[(1, 2), (1, 4), (2, 3), (2, 1), (4, 1), (4, 3), (3, 8), (8, 6), (6, 1), (6, 3), (7, 8), (7, 1), (7, 3)]


In [10]:
print(contractGraph(G, set([5, 6])).edges)

[(1, 2), (1, 4), (2, 3), (2, 1), (4, 1), (4, 3), (3, 8), (8, 1), (8, 3), (7, 8), (7, 1), (7, 3)]


In [11]:
R = {}

for v in G.nodes():
    R[v] = "UM"
    
# R[2] = "LM"
# R[3] = "LMD"
# R[5] = "RM"
# R[7] = "RMD"

In [12]:
def V_L(G, R):
    t = set()
    
    for v in G.nodes():
        if R[v] == "LM" or R[v] == "LMD":
            t.add(v)
    
    return t

def V_R(G, R):
    t = set()
    
    for v in G.nodes():
        if R[v] == "RM" or R[v] == "RMD":
            t.add(v)
    
    return t

In [13]:
print(V_L(G, R))

set()


In [14]:
def convertArb(R, F, N):
    for j in range(N):
        if F == "VR":
            for key in R:
                if R[key] == "RM" or R[key] == "RMD":
                    R[key] = "WRM"
                    break
        
        elif F == "VL":
            for key in R:
                if R[key] == "LM" or R[key] == "LMD":
                    R[key] = "WLM"
                    break

In [15]:
# convertArb(R, "VR", 1)
# print(R)

In [16]:
def balance(G, R):
    
    lenL = len(V_L(G, R))
    lenR = len(V_R(G, R))
    
    if lenR - lenL > 3:
        convertArb(R, "VR", lenR - lenL - 3)
    elif lenL - lenR > 3:
        convertArb(R, "VL", lenL - lenR - 3)

In [17]:
def checkcycle2(G):
    # Return any vertex of the first cycle of length 2.
    for v in list(G.nodes):
        l = G.out_edges(v)
        ln = [i[1] for i in l]
        
        for u in ln:
            l1 = G.out_edges(u)
            l1n = [i[1] for i in l1]
            
            if v in l1n:
                # Can return v or u here.
                return v
            
    return -1

In [18]:
def powerset(s):
    x = len(s)
    masks = [1 << i for i in range(x)]
    for i in range(1 << x):
        yield set([ss for mask, ss in zip(masks, s) if i & mask])

In [19]:
ww = list(powerset([4, 5, 6, 7]))
print(ww)
ww.remove(set([4,5]))
print(ww)

[set(), {4}, {5}, {4, 5}, {6}, {4, 6}, {5, 6}, {4, 5, 6}, {7}, {4, 7}, {5, 7}, {4, 5, 7}, {6, 7}, {4, 6, 7}, {5, 6, 7}, {4, 5, 6, 7}]
[set(), {4}, {5}, {6}, {4, 6}, {5, 6}, {4, 5, 6}, {7}, {4, 7}, {5, 7}, {4, 5, 7}, {6, 7}, {4, 6, 7}, {5, 6, 7}, {4, 5, 6, 7}]


In [20]:
wg = list(powerset([4, 5, 6]))
print(wg)

[set(), {4}, {5}, {4, 5}, {6}, {4, 6}, {5, 6}, {4, 5, 6}]


In [21]:
# G.remove_node(5)

In [22]:
try:
    nx.find_cycle(G)
    print("found one")
except:
    print("not found")

# print(list(nx.find_cycle(G)))

found one


In [23]:
def getAcyclicSubsets(G, s):
    # Returns a list of sets, containing all the acyclic subsets of the set s.
    
    allsets = list(powerset(s))
    
    for sub in allsets:
        if len(sub) == 0 or len(sub) == 1:
            continue
        
        G1 = G.subgraph(list(sub))
        
        try:
            nx.find_cycle(G1)
            allsets.remove(sub)
        except:
            pass
        
    return allsets

In [24]:
print(nx.number_strongly_connected_components(G))
print(list(nx.strongly_connected_components(G)))
print(type(list(nx.strongly_connected_components(G))[0]))
print(list(nx.strongly_connected_components_recursive(G)))

2
[{1, 2, 3, 4, 5, 6, 8}, {7}]
<class 'set'>
[{1, 2, 3, 4, 5, 6, 8}, {7}]


In [25]:
print(set(G.nodes))

{1, 2, 3, 4, 5, 6, 7, 8}


In [26]:
def GetMAS(G, R):
    # Returns set of vertices which form the MAS, ie. complement of FVS.
       
    # Balancing.
    balance(G, R)
    
    
    # C1
    # Return the MAS directly and return it.
    l = list(G.nodes)
    
    if len(l) == 1 or len(l) == 0:
        return set(G.nodes)
    
    elif len(l) == 2:
        try:
            nx.find_cycle(G)
            return set(list(G.nodes)[0])
        except:
            return set(G.nodes)
    
    if len(l) == 3:
        try:
            nx.find_cycle(G)
            
            ps = list(powerset(l))
            for s in ps:
                if len(s) == 2:
                    Gm = G.subgraph(list(s))
                    try:
                        nx.find_cycle(Gm)
                    except:
                        return s
                                    
            for s in ps:
                if len(s) == 1:
                    return s
        except:
            return set(G.nodes)

        
    # C2
    t = checkcycle2(G)
    if t != -1:
        if R[t] == "UM":
            R1 = copy.deepcopy(R)
        elif R[t] == "LM" or R[t] == "LMD" or R[t] == "WLM":
            R1 = copy.deepcopy(R)
            e = G.in_edges(t)
            en = [i[0] for i in e]
            for k in R1:
                if k in en and R1[k] == "UM":
                    R1[k] = "WLM"
        elif R[t] == "RM" or R[t] == "RMD" or R[t] == "WRM":
            R1 = copy.deepcopy(R)
            l = G.out_edges(t)
            ln = [i[1] for i in l]
            for k in R1:
                if k in ln and R1[k] == "UM":
                    R1[k] = "WRM"

        S1 = GetMAS(G, R)
        S2 = GetMAS(contractGraph(G, t), R1) | set([t])
        
        if len(S1) > len(S2):
            return S1
        else:
            return S2
        
    # C3
    # If the graph has at least 2 scc.
    # Get 2 vertices, v1 and v2 from 2 SCC G1 and G2, that are preferably LMD or RMD.
    
    allscc = list(nx.strongly_connected_components(G))
    
    if len(allscc) >= 2:
        # G1 and G2 are the SCC components.
        G1 = G.subgraph(list(allscc[0]))
        G2 = G.subgraph(list(allscc[1]))
        
        v1 = list(G1.nodes)[0]
        v2 = list(G2.nodes)[0]
        
        for i in list(G1.nodes):
            if R[i] == "LMD" or R[i] == "RMD":
                v1 = i
                break
        
        for i in list(G2.nodes):
            if R[i] == "LMD" or R[i] == "RMD":
                v2 = i
                break
        
        t1 = set([v1,v2]) | GetMAS(contractGraph(G, set([v1,v2])), R)
        # Gn.remove_node(v1)
        # Gn.remove_node(v2)
        rl = list(G.nodes)
        rl.remove(v1)
        rl.remove(v2)
        t2 = GetMAS(G.subgraph(rl), R)
        
        x1 = t1 & set(G1.nodes)
        x2 = t2 & set(G1.nodes)
        if len(x1) > len(x2):
            xm = copy.deepcopy(x1)
        else:
            xm = copy.deepcopy(x2)
            
        y1 = t1 & set(G2.nodes)
        y2 = t2 & set(G2.nodes)
        
        if len(y1) > len(y2):
            ym = copy.deepcopy(y1)
        else:
            ym = copy.deepcopy(y2)
        
        z1 = set(G1.nodes) | set(G2.nodes)
        z = t1 - z1
        
        b = xm | ym | z
        
        return b
    
    # C4 !!
    if len(V_L(G,R)) == 0 or len(V_R(G,R)) == 0:
        
        # C41
        for v in list(G.nodes):
            if R[v] == "UM" or R[v] == "WLM" or R[v] == "WRM":
                e = G.in_edges(v)
                en = [i[0] for i in e]
                l = G.out_edges(v)
                ln = [i[1] for i in l]
                rn = list(G.nodes)
                
                if len(en) == 0:
                    return set([v]) | GetMAS(contractGraph(G, v), R)
                
                elif len(en) == 1:
                    rl = list(G.nodes)
                    rl.remove(v)
                    return max(set([v]) | GetMAS(contractGraph(G, v), R), set([en[0]]) | GetMAS(contractGraph(G.subgraph(rl), en[0]), R), key=len)
                
                elif len(en) == 2:
                    rl = list(G.nodes)
                    r1 = rl.remove(v)
                    r2 = rl.remove(en[0])
                    return max(set([v]) | GetMAS(contractGraph(G, v), R), set([en[0]]) | GetMAS(contractGraph(G.subgraph(r1), en[0]), R), set([en[1]]) | GetMAS(contractGraph(G.subgraph(r2), en[1]), R), key=len)
                
                elif len(en) == 3:
                    rl = list(G.nodes)
                    r1 = rl.remove(v)
                    r2 = rl.remove(en[0])
                    r3 = rl.remove(en[1])
                    return max(set([v]) | GetMAS(contractGraph(G, v), R), set([en[0]]) | GetMAS(contractGraph(G.subgraph(r1), en[0]), R), set([en[1]]) | GetMAS(contractGraph(G.subgraph(r2), en[1]), R), set([en[2]]) | GetMAS(contractGraph(G.subgraph(r3), en[2]), R), key=len)
                
                elif len(ln) == 0:
                    return set([v]) | GetMAS(contractGraph(G, v), R)
                
                elif len(ln) == 1:
                    rl = list(G.nodes)
                    rl.remove(v)
                    return max(set([v]) | GetMAS(contractGraph(G, v), R), set([ln[0]]) | GetMAS(contractGraph(G.subgraph(rl), ln[0]), R), key=len)
                
                elif len(ln) == 2:
                    rl = list(G.nodes)
                    r1 = rl.remove(v)
                    r2 = rl.remove(ln[0])
                    return max(set([v]) | GetMAS(contractGraph(G, v), R), set([ln[0]]) | GetMAS(contractGraph(G.subgraph(r1), ln[0]), R), set([ln[1]]) | GetMAS(contractGraph(G.subgraph(r2), ln[1]), R), key=len)
                
                elif len(ln) == 3:
                    rl = list(G.nodes)
                    r1 = rl.remove(v)
                    r2 = rl.remove(ln[0])
                    r3 = rl.remove(ln[1])
                    return max(set([v]) | GetMAS(contractGraph(G, v), R), set([ln[0]]) | GetMAS(contractGraph(G.subgraph(r1), ln[0]), R), set([ln[1]]) | GetMAS(contractGraph(G.subgraph(r2), ln[1]), R), set([ln[2]]) | GetMAS(contractGraph(G.subgraph(r3), ln[2]), R), key=len)

        # C42
        for v in list(G.nodes):
            if R[v] == "UM" or R[v] == "WLM" or R[v] == "WRM":
                R1 = copy.deepcopy(R)
                
                e = G.in_edges(v)
                en = [i[0] for i in e]
                l = G.out_edges(v)
                ln = [i[1] for i in l]
                
                tok = 4
                for i in en:
                    if tok != 0:
                        R1[i] = "LM"
                        tok = tok - 1
                    else:
                        R1[i] = "WLM"
                tok = 4
                for i in ln:
                    if tok != 0:
                        R1[i] = "RM"
                        tok = tok - 1
                    else:
                        R1[i] = "WRM"
                
                rl = list(G.nodes)
                rl.remove(v)
                
                return max(set([v]) | GetMAS(contractGraph(G, v), R1), GetMAS(G.subgraph(rl)), key=len)
                
        R1 = copy.deepcopy(R)
        
        e = G.in_edges(v)
        en = [i[0] for i in e]
        l = G.out_edges(v)
        ln = [i[1] for i in l]
        
        tok = 4
        for i in en:
            if tok != 0:
                R1[i] = "LM"
                tok = tok - 1
            else:
                R1[i] = "WLM"
        tok = 4
        for i in ln:
            if tok != 0:
                R1[i] = "RM"
                tok = tok - 1
            else:
                R1[i] = "WRM"
        
        rl = list(G.nodes)
        rl.remove(v)
        
        return max(set([v]) | GetMAS(contractGraph(G, v), R1), GetMAS(G.subgraph(rl)), key=len)

    # C5
    
    lenL = len(V_L(G, R))
    lenR = len(V_R(G, R))
    
    # For the first part.
    if(lenL < lenR):
        # C51 not needed.
        # C52
        for v in list(G.nodes):
            if R[v] == "LM" or R[v] == "LMD":
                e = G.in_edges(v)
                en = [i[0] for i in e]
                
                uen = [i for i in en if R[i] == "UM"]
                
                if len(uen) >= 4:
                    R1 = copy.deepcopy(R)
                    tok = 4
                    for j in uen:
                        if tok != 0:
                            R1[j] = "LM"
                            tok = tok - 1
                        else:
                            R1[j] = "WLM"
                    
                    rl = list(G.nodes)
                    rl.remove(v)
                    
                    return max(set([v]) | GetMAS(contractGraph(G, v), R1), GetMAS(G.subgraph(rl), R), key=len)
                    
        # C53
        ue = []
        v = 0
        for vt in list(G.nodes):
            e = G.in_edges(vt)
            en = [i[0] for i in e]
            
            uet = [i for i in en if R[i] == "UM"]
            
            if len(uet) > len(ue):
                # ue = uet.copy()
                ue = uet
                v = vt
        
        # ue is X.
        # Ys is a set of all Y.
        Ys = getAcyclicSubsets(G, ue)
        Ty = []
        
        for Y in Ys:
            Rt = copy.deepcopy(R)
            if(len(Y) == 0):
                Rt[v] = "LMD"
            
            temp = Y | GetMAS(contractGraph(G.subgraph(list(ue - Y)), Y), Rt)
            Ty.append(temp)
            
        return max(Ty, key=lambda item: len(item))
    
    # For the second part.
    else:
        # C51 not needed.
        # C52
        for v in list(G.nodes):
            if R[v] == "RM" or R[v] == "RMD":
                l = G.out_edges(v)
                ln = [i[1] for i in l]
                
                uln = [i for i in ln if R[i] == "UM"]
                
                if len(uln) >= 4:
                    R1 = copy.deepcopy(R)
                    tok = 4
                    for j in uln:
                        if tok != 0:
                            R1[j] = "RM"
                            tok = tok - 1
                        else:
                            R1[j] = "WRM"
                    
                    rl = list(G.nodes)
                    rl.remove(v)
                    
                    return max(set([v]) | GetMAS(contractGraph(G, v), R1), GetMAS(G.subgraph(rl), R), key=len)
                    
        # C53
        ul = []
        v = 0
        for vt in list(G.nodes):
            l = G.out_edges(vt)
            ln = [i[1] for i in l]
            
            ult = [i for i in ln if R[i] == "UM"]
            
            if len(ult) > len(ul):
                # ul = ult.copy()
                ul = ult
                v = vt
        
        Ys = getAcyclicSubsets(G, v)
        Ty = []
        
        for Y in Ys:
            Rt = copy.deepcopy(R)
            if(len(Y) == 0):
                Rt[v] = "RMD"
            
            temp = Y | GetMAS(contractGraph(G.subgraph(list(ul - Y)), Y), Rt)
            Ty.append(temp)
            
        return max(Ty, key=lambda item: len(item))
                
   

In [27]:
ff = list(G.nodes)
print(type(ff))
ff.remove(8)
print(ff)

hh = G.subgraph(ff)
print(G)
print(hh)

<class 'list'>
[1, 2, 4, 3, 5, 6, 7]
DiGraph with 8 nodes and 12 edges
DiGraph with 7 nodes and 9 edges


In [28]:
print(R)
e = G.in_edges(5)
en = [i[0] for i in e]
print(en)
uen = [i for i in en if R[i] == "UM"]
print(uen)

{1: 'UM', 2: 'UM', 4: 'UM', 3: 'UM', 5: 'UM', 8: 'UM', 6: 'UM', 7: 'UM'}
[2, 4, 6, 7]
[2, 4, 6, 7]


In [29]:
for i in en:
    print(R[i])

UM
UM
UM
UM


In [30]:
print(set([8]))

{8}


In [31]:
tt = [set([1,2]), set([11,22, 3]), set([133, 44 , 4 , 2]), set([14]), set([15, 6, 7, 3, 87, 75, 2]), set([]), set([13, 4])]

print(max(tt, key=lambda item: len(item)))

{2, 3, 6, 7, 75, 15, 87}


In [34]:
g = nx.DiGraph()

g.add_edges_from(
    [
        (1, 2),
        (2, 3),
        (3, 4),
        (3, 1),
        (4, 1),
        (2, 5)
    ]
)

r = {}

for v in G.nodes():
    r[v] = "UM"

In [35]:
print(GetMAS(g, r))

{2, 3, 4, 5}
